In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('retina')

In [2]:
from __future__ import print_function, division
import os,sys
import numpy as np
import torch # pytorch package, allows using GPUs
# fix seed
seed=17
np.random.seed(seed)
torch.manual_seed(seed)
from datetime import datetime
from glob import glob
import pandas as pd
import h5py
import psutil

In [3]:
mem_usage = psutil.virtual_memory()
mem_usage_start = mem_usage.used

# Step 1. Identify data and info

In [4]:
X_train_file_info = pd.DataFrame({'filelist' : glob('data/tmax_train/*.h5')})

In [5]:
X_train_file_info['num'] = [int(file.split('/')[2].split('_')[-1].split('.')[0]) for file in X_train_file_info['filelist']]

In [6]:
X_train_file_info = X_train_file_info.sort_values('num')
X_train_file_info.index = range(len(X_train_file_info))

In [7]:
X_train_info = np.load('data/tmax_train/tmax_X_train_info.npz').get('arr_0')

In [8]:
y_train = np.load('data/tmax_train/tmax_y_train.npz', allow_pickle=True).get('arr_0')

In [9]:
mem_usage = psutil.virtual_memory()

print(f"Free: {mem_usage.percent}%")
print(f"Total: {mem_usage.total/(1024**3):.2f}G")
print(f"Used: {mem_usage.used/(1024**3):.2f}G")
print(f"Used - Start: {(mem_usage.used - mem_usage_start)/(1024**3):.2f}G")

Free: 52.1%
Total: 187.37G
Used: 96.77G
Used - Start: 0.27G


In [10]:
X_val_file_info = pd.DataFrame({'filelist' : glob('data/tmax_val/*.h5')})

In [11]:
X_val_file_info['num'] = [int(file.split('/')[2].split('_')[-1].split('.')[0]) for file in X_val_file_info['filelist']]

In [12]:
X_val_file_info = X_val_file_info.sort_values('num')
X_val_file_info.index = range(len(X_val_file_info))

In [13]:
X_val_info = np.load('data/tmax_val/tmax_X_val_info.npz').get('arr_0')

In [14]:
y_val = np.load('data/tmax_val/tmax_y_val.npz', allow_pickle=True).get('arr_0')

In [15]:
mem_usage = psutil.virtual_memory()

print(f"Free: {mem_usage.percent}%")
print(f"Total: {mem_usage.total/(1024**3):.2f}G")
print(f"Used: {mem_usage.used/(1024**3):.2f}G")
print(f"Used - Start: {(mem_usage.used - mem_usage_start)/(1024**3):.2f}G")

Free: 52.1%
Total: 187.37G
Used: 96.80G
Used - Start: 0.30G


# Step 2. Initialize the dataset with a data loader

In [16]:
import torch
from torch.utils.data import Dataset, DataLoader

class WeatherDataset(Dataset):
    def __init__(self, data_path, y_data, batch_size=1000):
        super().__init__()
        with h5py.File(data_path, 'r') as f:
            self.data_X_train = np.array(f['data'])
        self.data_y_train = np.array(np.split(y_data, len(self.data_X_train)//batch_size))
        self.data_X_train = np.array(np.split(self.data_X_train, len(self.data_X_train)//batch_size))
        
    def __getitem__(self, idx):
        return self.data_X_train[idx]

    def __len__(self):
        return len(self.data_X_train)

def load_data(dataset_path, y_data, i):
    # define dataset path
    
    # create dataset object
    batch_size = 1000
    dataset = WeatherDataset(dataset_path[i], y_data[i*1000:i*1000+1000], batch_size=batch_size)

    # create data loader object
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return data_loader

In [17]:
data_chunk = load_data(X_train_file_info['filelist'].tolist(), y_train, 0)

In [18]:
data_chunk.dataset.data_X_train.shape

(1, 1000, 10, 11, 365)

In [19]:
data_chunk.dataset.data_y_train.shape

(1, 1000, 7)

In [20]:
mem_usage = psutil.virtual_memory()

print(f"Free: {mem_usage.percent}%")
print(f"Total: {mem_usage.total/(1024**3):.2f}G")
print(f"Used: {mem_usage.used/(1024**3):.2f}G")
print(f"Used - Start: {(mem_usage.used - mem_usage_start)/(1024**3):.2f}G")

Free: 52.2%
Total: 187.37G
Used: 97.12G
Used - Start: 0.61G


# Step 3. Build a simple architecture to explore

In [136]:
import torch.nn as nn # construct NN
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        # 3D convolutional layers
        self.conv1 = nn.Conv3d(in_channels=10, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv3d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        
        # Batch normalization layers
        self.bn1 = nn.BatchNorm3d(num_features=16)
        self.bn2 = nn.BatchNorm3d(num_features=32)
        
        # Max pooling layer
        self.pool = nn.MaxPool3d(kernel_size=(1,2,2), stride=(1,2,2))
        
        # Fully connected layers
        self.fc1 = nn.Linear(in_features=32*11*91*250, out_features=512) #32*5*182*128
        self.fc2 = nn.Linear(in_features=512, out_features=7)
        
        # Dropout layer
        #self.dropout = nn.Dropout(p=0.5)
        
        # ReLU activation function
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # Input shape: (batch_size=400000, channels=10, depth=11, height=365, width=1)
        
        # First convolutional block
        #print(x.shape)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        #print(x.shape)
        
        # Second convolutional block
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool(x)
        #print(x.shape)
        
        # Flatten
        x = torch.flatten(x, start_dim=1)
        #print(x.shape)
        
        # Fully connected layers with dropout
        x = self.fc1(x)
        x = self.relu(x)
        #x = self.dropout(x)
        x = self.fc2(x)
        
        # Output shape: (batch_size=400000, num_classes=10)
        return x


In [173]:
import torch.optim as optim

def train(model, train_loader, val_loader, num_epochs=10, learning_rate=0.001):
    # Set device to GPU if available, else CPU
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = torch.device("cpu")
    model.to(device)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        
        '''
        # iterate over the batches
        for batch_i in range(train_loader.dataset.data_X_train.shape[1]):
            inputs = torch.Tensor(train_loader.dataset.data_X_train[:, batch_i, :, :, :].astype('int64')).reshape(10, 11, 365, 1)
            labels = torch.Tensor(train_loader.dataset.data_y_train[:, batch_i, :].astype('int64'))

            # Move data to device
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Track training loss and accuracy
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_correct += (predicted == labels).sum().item()
        '''
        inputs = torch.Tensor(train_loader.dataset.data_X_train.astype('int64').reshape(1, 10, 11, 365, 1000))
        labels = torch.Tensor(train_loader.dataset.data_y_train.astype('int64')[0, :, :].reshape(7, 1000))
        # Move data to device
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        print(outputs.shape)
        print(labels.shape)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track training loss and accuracy
        train_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        
        train_correct += (predicted == labels).sum().item()

        
        # Calculate average training loss and accuracy
        train_loss /= len(train_loader.dataset)
        train_accuracy = 100. * train_correct / len(train_loader.dataset)
        
        # Evaluate on validation set
        model.eval()
        val_loss = 0.0
        val_correct = 0
        
        # Disable gradient computation
        with torch.no_grad():
            '''
            # Iterate over val loader
            for inputs, labels in zip(val_loader.dataset.data_X_train, val_loader.dataset.data_y_train):
                # Move data to device
                inputs, labels = inputs.to(device), labels.to(device)
                
                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                # Track validation loss and accuracy
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_correct += (predicted == labels).sum().item()
            '''
            
            inputs = torch.Tensor(val_loader.dataset.data_X_train.astype('int64').reshape(1, 10, 11, 365, 1000))
            labels = torch.Tensor(val_loader.dataset.data_y_train.astype('int64')[0, :, :].reshape(7, 1000))
            # Move data to device
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Track validation loss and accuracy
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_correct += (predicted == labels).sum().item()
                
        # Calculate average validation loss and accuracy
        val_loss /= len(val_loader.dataset)
        val_accuracy = 100. * val_correct / len(val_loader.dataset)
        
        # Print epoch statistics
        print("Epoch [{}/{}], Train Loss: {:.4f}, Train Acc: {:.2f}%, Val Loss: {:.4f}, Val Acc: {:.2f}%"
              .format(epoch+1, num_epochs, train_loss, train_accuracy, val_loss, val_accuracy))


In [174]:
def train_loader(i):
    return load_data(X_train_file_info['filelist'].tolist(), y_train, i)

In [175]:
def val_loader(i):
    return load_data(X_val_file_info['filelist'].tolist(), y_val, i)

In [176]:
model = Model()

In [177]:
mem_usage = psutil.virtual_memory()

print(f"Free: {mem_usage.percent}%")
print(f"Total: {mem_usage.total/(1024**3):.2f}G")
print(f"Used: {mem_usage.used/(1024**3):.2f}G")
print(f"Used - Start: {(mem_usage.used - mem_usage_start)/(1024**3):.2f}G")

Free: 73.9%
Total: 187.37G
Used: 137.73G
Used - Start: 41.22G


In [178]:
train(model, train_loader(0), val_loader(0), num_epochs=1)

torch.Size([1, 7])
torch.Size([7, 1000])


ValueError: Expected input batch_size (1) to match target batch_size (7).

In [ ]:
mem_usage = psutil.virtual_memory()

print(f"Free: {mem_usage.percent}%")
print(f"Total: {mem_usage.total/(1024**3):.2f}G")
print(f"Used: {mem_usage.used/(1024**3):.2f}G")
print(f"Used - Start: {(mem_usage.used - mem_usage_start)/(1024**3):.2f}G")

In [98]:
import torch.nn as nn # construct NN

class model(nn.Module):
    # create convolutional net
    def __init__(self, N=10, L=40):
        # inherit attributes and methods of nn.Module
        super(model, self).__init__()	
        # create convolutional layer with input depth 1 and output depth N
        self.conv1 = nn.Conv3d(10, N, kernel_size=3, padding=1)
        # batch norm layer takes Depth
        self.bn1=nn.BatchNorm3d(N) 
        # create fully connected layer after maxpool operation reduced 40->18
        self.fc1 = nn.Linear(1000, 7) 	
        self.N=N
        self.L=L
        print("The number of neurons in CNN layer is %i"%(N))

    def forward(self, x):
        x = torch.Tensor(x)
        #print(x.shape)
        x = F.relu(self.conv1(x))
        #print(x.shape)  often useful to look at shapes for debugging
        x = F.max_pool3d(x,3)
        #print(x.shape)
        x=self.bn1(x) # largely unnecessary and here just for pedagogical purposes
        return F.log_softmax(self.fc1(x.view(-1,20*20*self.N)), dim=1)

In [99]:
def train(train_loader, epoch):
    CNN.train() # effects Dropout and BatchNorm layers
    data = train_loader.dataset.data_X_train
    target = train_loader.dataset.data_y_train
    
    optimizer.zero_grad()
    output = CNN(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % args.log_interval == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, 0 * len(data), len(train_loader.dataset),
            100. * 0 / len(train_loader), loss.item()))

In [100]:
def test(data_loader,verbose='Test'):
    # these are very standard functions for evaluating data

    CNN.eval() # effects Dropout and BatchNorm layers
    test_loss = 0
    correct = 0
    for data, target in data_loader:
        output = CNN(data)
        test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    test_loss /= len(data_loader.dataset)
    print('\n'+verbose+' set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))
    accuracy=100. * correct / len(data_loader.dataset)
    return(accuracy)

In [102]:
import torch.nn.functional as F # implements forward and backward definitions of an autograd operation
import torch.optim as optim # different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc

CNN = model(N=1)
# negative log-likelihood (nll) loss for training: takes class labels NOT one-hot vectors!
criterion = F.nll_loss
# define optimizer
optimizer = optim.SGD(CNN.parameters(), lr=0.001, momentum=0.001)
#optimizer = optim.Adam(DNN.parameters(), lr=0.001, betas=(0.9, 0.999))

train(train_loader(0), 1)

The number of neurons in CNN layer is 1


RuntimeError: Given groups=1, weight of size [1, 10, 3, 3, 3], expected input[1, 1000, 10, 11, 365] to have 10 channels, but got 1000 channels instead

In [ ]:
train_loader, test_loader, critical_loader=load_data(cuda_kwargs)

next(iter(train_loader))[0].shape

In [ ]:
epochs = 10
lr = 0.001

In [ ]:
test_array=[]
critical_array=[]

# create array of depth of convolutional layer
N_array=[1]

# loop over depths
for N in N_array:
    CNN = model(N=N)

    # negative log-likelihood (nll) loss for training: takes class labels NOT one-hot vectors!
    criterion = F.nll_loss
    # define optimizer
    #optimizer = optim.SGD(CNN.parameters(), lr=args.lr, momentum=args.momentum)
    optimizer = optim.Adam(DNN.parameters(), lr=0.001, betas=(0.9, 0.999))

    # train the CNN and test its performance at each epoch
    for epoch in range(1, epochs + 1):
        train(epoch)
        if epoch==args.epochs:
            test_array.append(test(test_loader,verbose='Test'))
        else:
            test(test_loader,verbose='Test')
    print(test_array)
    print(critical_array)